indicator that makes that 0:00 H3 bullish bar blue and red if bearish

#property indicator_chart_window
#property indicator_buffers 5
#property indicator_plots   1

#property indicator_type1   DRAW_COLOR_CANDLES
#property indicator_label1  "H3 00:00 Candle"
#property indicator_color1  clrBlue, clrRed

// =====================
// BUFFERS
// =====================
double OpenBuf[];
double HighBuf[];
double LowBuf[];
double CloseBuf[];
double ColorBuf[];

// =====================
// INIT
// =====================
int OnInit()
{
   SetIndexBuffer(0, OpenBuf,  INDICATOR_DATA);
   SetIndexBuffer(1, HighBuf,  INDICATOR_DATA);
   SetIndexBuffer(2, LowBuf,   INDICATOR_DATA);
   SetIndexBuffer(3, CloseBuf, INDICATOR_DATA);
   SetIndexBuffer(4, ColorBuf, INDICATOR_COLOR_INDEX);

   PlotIndexSetInteger(0, PLOT_DRAW_BEGIN, 0);
   PlotIndexSetInteger(0, PLOT_COLOR_INDEXES, 2);

   IndicatorSetString(INDICATOR_SHORTNAME, "H3 00:00 Bull/Bear Candle");

   return(INIT_SUCCEEDED);
}

// =====================
// CALCULATION
// =====================
int OnCalculate(
   const int rates_total,
   const int prev_calculated,
   const datetime &time[],
   const double &open[],
   const double &high[],
   const double &low[],
   const double &close[],
   const long &tick_volume[],
   const long &volume[],
   const int &spread[]
)
{
   // Only valid on H3
   if(_Period != PERIOD_H3)
      return(rates_total);

   int start = (prev_calculated > 0) ? prev_calculated - 1 : 0;

   for(int i = start; i < rates_total; i++)
   {
      // Copy price data
      OpenBuf[i]  = open[i];
      HighBuf[i]  = high[i];
      LowBuf[i]   = low[i];
      CloseBuf[i] = close[i];

      ColorBuf[i] = EMPTY_VALUE;

      MqlDateTime t;
      TimeToStruct(time[i], t);

      // Check H3 candle starting at 00:00
      if(t.hour == 0 && (t.hour % 3) == 0)
      {
         if(close[i] > open[i])
            ColorBuf[i] = 0; // Blue (bullish)
         else if(close[i] < open[i])
            ColorBuf[i] = 1; // Red (bearish)
      }
   }

   return(rates_total);
}
